In [0]:
data = [
  (1, "Arjun", "M", "Chennai", 50000),
  (2, "Ravi", "M", "Bangalore", 60000),
  (3, "Meena", "F", None, 70000),
  (4, "Raj", "M", "Delhi", 80000)
]

column = ["Id", "Name","Gender", "City", "Salary"]

df = spark.createDataFrame(data, column)

In [0]:
display(df)

In [0]:
device_data = [
    (101,"Laptop" ,"Dell", 50000),
    (102,"Laptop", "HP", 45000),
    (103,"Laptop", "Lenovo", 40000),
    (104, "SmartPhone", "Redmi", 15000)
]

device_column = ["Id", "Product", "Brand", "Price"]

df_device = spark.createDataFrame(device_data, device_column)
df_device.show()


In [0]:
# Filter() is like where in sql
from pyspark.sql.functions import col

df_device.filter(col("Price")>40000).show()

In [0]:
df_device.where(col("Price")>40000).show()


In [0]:
df_device.filter(
    (col("Product") == "Laptop") & (col("Price") >= 45000) 
).show()

In [0]:
# To add fixed values inside expressions.
from pyspark.sql.functions import lit

df_flag = df_device.withColumn("Country", lit("India"))
df_flag.show()

In [0]:
df_flag.filter(col("Brand") != lit("HP")).show()
# without lit(), spark can't understand raw constants

In [0]:
# when() is like case when in sql
# otherwise() is like else in sql
# when() / otherwise() -- if else of spark

from pyspark.sql.functions import when

df_segment = df_flag.withColumn(
    "Price_category",
    when(col("Price") >= 45000, "Premium").otherwise("Budget") 
)
df_segment.show()

In [0]:
df_segment = df_flag.withColumn(
    "Price_category",
    when(col("Price") >= 50000, "High").
    when(col("Price") >= 40000, "Medium").
    otherwise("Low")
)
# when/otherwise is spark's contional expression evaluated lazily.
df_segment.show()

In [0]:
df_segment.filter(col("Brand").isNull()).show()

In [0]:
df_segment.filter(col("Brand").isNotNull()).show()

In [0]:
# expr() allows SQL-style expressions inside the dataFrame API.
from pyspark.sql.functions import expr

df_expr = df_segment.withColumn(
    "Discounted_price",
    expr("Price * 0.9")
)
df_expr.show()

In [0]:
df_expr.filter(col("Brand").isin("Dell","HP")).show()

In [0]:
df_expr.where(col("Brand").isin("Dell","HP")).show()

In [0]:
df_Festival = df_expr.withColumn(
    "Offer_type",
    when(col("Price") >= 45000, "Big Deal").
    otherwise("Normal Deal")
)

df_Festival.show()

In [0]:
df_Festival.filter(col("Brand") != lit("HP"))\
    .select("Product","Brand","Offer_type").show()

In [0]:
display(df_Festival)

In [0]:
df_Festival.write.mode("overwrite").saveAsTable("dev_data.test.Products")

In [0]:
%sql
select * from dev_data.test.products